# Creating animations from DANRA

Updated: 2024-02-29, Leif Denby (lcd@dmi.dk)

This notebook is an example of how you can easily create animations from DANRA data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import io
import imageio.v2 as imageio
from tqdm import tqdm

from pathlib import Path
import xarray as xr

In [2]:
version = "v0.3.0"
# path to copies of processed DANRA datasets on scale.dmi.dk:
# fp_root = Path(f"/dmidata/projects/cloudphysics/danra/data/{version}")
# path on ohm.dmi.dk where the files where processed:
fp_root = Path(f"/nwp/danra/data/{version}")

In [3]:
ds_sl = xr.open_zarr(fp_root / "single_levels.zarr")

In [4]:
ds = ds_sl
# let's add a 2m temperature in celcius to work with
ds["t2m_celcius"] = ds.t2m - 273.15
ds.t2m_celcius.attrs["units"] = "C"
ds.t2m_celcius.attrs["long_name"] = ds.t2m.long_name

In [8]:
temp_max = 30.0
tmin = ds.time.isel(time=0)
tmax = tmin + np.timedelta64(365, "D")
DATE_FORMAT = "%Y%m%d"
writer = imageio.get_writer(f'DANRA_t2m_{tmin.dt.strftime(DATE_FORMAT).item()}-{tmax.dt.strftime(DATE_FORMAT).item()}.mp4', fps=10)

import datetime

# Example: Generate 10 frames
da = ds.t2m_celcius.sel(time=slice(tmin, tmax))
da.attrs["long_name"] = f"2m {da.long_name}"
for i in tqdm(list(range(int(da.time.count())))):
    # Create a figure
    fig, ax = plt.subplots(figsize=(10, 4), subplot_kw=dict(projection=ccrs.PlateCarree()))
    da_sample = da.isel(time=i)
    da_sample.plot(ax=ax, x="lon", y="lat", transform=ccrs.PlateCarree(), vmax=temp_max, center=0.0, cmap="RdBu_r", extend="both")
    ax.gridlines(draw_labels=["top", "left"])
    ax.coastlines()
    
    t = da_sample.time.data.astype("datetime64[s]").astype(datetime.datetime).item()

    ax.set_title(t.isoformat())
    fig.tight_layout()
    
    # Use io.BytesIO as a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=400)  # Save figure to the buffer
    buf.seek(0)  # Move to the beginning of the buffer
    
    # Read the buffer's content as an image
    image = imageio.imread(buf)
    
    # Add image to the writer (i.e., to the animation)
    writer.append_data(image)
    
    # Close the figure to prevent it from being displayed
    plt.close(fig)

writer.close()

100%|██████████| 25/25 [00:39<00:00,  1.57s/it]


In [7]:
writer.close()